In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction
In this notebook, we will show you how to build a Dialogflow CX Agent from scratch using plain text inputs and a simple CSV file.

In this example some of the training phrases will be anotated with the @sys.number entity

## Prerequisites
- Ensure you have a GCP Service Account key with the Dialogflow API Admin privileges assigned to it.

In [ ]:
#If you haven't already, make sure you install the `dfcx-scrapi` library
#Link to the GitHub repo: https://github.com/GoogleCloudPlatform/dfcx-scrapi

!pip install --user dfcx-scrapi
!pip install --user -r ./dfcx-scrapi/requirements.txt

# Imports

In [ ]:
import pandas as pd
from dfcx_scrapi.core.agents import Agents
from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.core.flows import Flows
from dfcx_scrapi.core.pages import Pages
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions
from dfcx_scrapi.tools.maker_util import MakerUtil

#Input here your Service Account Key path
creds_path = '/PATH/TO/YOU/KEY.JSON'

# Method 1 - Creating an Agent from Simple Text Inputs

## Create Your Agent
Creating an agent requires a minimum of 2 pieces of information:
- `project_id`, which is your GCP Project ID
- `display_name`, (i.e. 'My Cool Agent!')
- `gcp_region`, (Optional) This defaults to `global` region, but you can provide any GCP region that is currently available for Dialogflow CX.

In [ ]:
# First we will instantiate our Agent object
a = Agents(creds_path=creds_path)

# Next, we will set some variables for our agent creation or retrieval args
project_id = '<PROJECT_ID>'
display_name = 'My Awesome Agent!'
gcp_region = '<GCP_REGION>' #(eg. 'global', 'us-cenral1', etc... See region availability for DFCX at https://cloud.google.com/dialogflow/cx/docs/concept/region

# Then we will call the `create_agent` or get_agent method (if agent already exists) and capture the result in a var call `my_agent`
#Option 1: If agent does not exist
#my_agent = a.create_agent(project_id, display_name, gcp_region)

#Option 2: If agent already exists
#The agent ID must be entered as a string "projects/<PROJECT_ID>/locations/<GCP_REGION>/agents/<AGENT_ID>"
my_agent = a.get_agent("projects/<PROJECT_ID>/locations/<GCP_REGION>/agents/<AGENT_ID>")

If you inspect your `my_agent` variable, you will see it is of types.agent.Agent

In [ ]:
my_agent

## Create Your First Intent
For this demo agent, we'll build a basic intent from list of Training Phrases (TPs)

To simplify the Intent creation, we'll utilize the `DataframeFunctions` class from the `tools` portion of the SCRAPI library.   
This will allow us to build our intent into a simple Pandas DataFrame, and then push this DataFrame directly into our bot that we just created.

A common method of building Intents and Training Phrases for Dialogflow CX agents is to use Google Sheets or CSVs to store the Intent/TP data.  
For this demo, we are working with annotated training phrases. To do so, we've included 
* `intent_sample_with_parts.csv` which contains all the training phrases parts. It will be pulled into a dataframe 
* `intent_sample_params`, that defines the parameters used for each intent. It will also be pulled into a dataframe

In [ ]:
# First, we will instantiate our DataframeFunctions (dffx) object
dffx = DataframeFunctions(creds_path=creds_path)

# Next, we will read in our sample CSV with Intent training phrases data as well as the entities used into 2 distinct Pandas DataFrames
df = pd.read_csv('../../data/intent_sample_with_parts.csv')
params_df = pd.read_csv('../../data/intent_sample_params.csv')

# Finally, we will use `dffx` to push our Intents to our Agent
#If intents do not exist, use bulk_create_intent_from_dataframe
my_intents = dffx.bulk_create_intent_from_dataframe(my_agent.name, df, params_df, update_flag=True, mode="advanced")

#If intent already exist, use bulk_update_intents_from_dataframe
#my_intents = dffx.bulk_update_intents_from_dataframe(my_agent.name, df, params_df, update_flag=True, mode="advanced")

# Bot Building 102 End
## And there you have it!   
We've created a simple Dialogflow CX agent using only Python in a Jupyter notebook.
You can see how this could be easily scaled up using .py files, git repos, and other scripts to speed up the bot building process.